In [21]:
import pandas as pd

from PIL import Image
from pytesseract import pytesseract

import os

import string
import re

In [13]:
tess_path = "C:\Program Files\Tesseract-OCR"
pytesseract.tesseract_cmd = os.path.join(tess_path,'tesseract.exe')
img_path = "screenshots"
imgs = os.listdir(img_path)

In [32]:
usernames = ['gnome','andyvo','speckled mug','jack-o-pie','megh']
codes = list(string.ascii_lowercase)[:5] 

In [28]:
len(imgs)

160

In [56]:
# load images -> grab text
rows = []
for i,img in enumerate(imgs):
    
    if i % 10 == 0:
        print(i)

    # get text from image
    img = Image.open(os.path.join(img_path,img))
    text = pytesseract.image_to_string(img)
    # print(text,'\n')

    row = []
    # get date
    date = re.findall('day\s*(\D+\s{1}\d+,\s{1}\d+)',text)[0]
    row.append(date)

    # get completion times
    for un in usernames:
        completion_time = re.findall(un + '\s{1}\S*\s*(\d{2}:\d{2})',text)[0] if  re.findall(un + '\s{1}\S*\s*(\d{2}:\d{2})',text) else None
        row.append(completion_time)
    rows.append(row)

rows[:2]

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150


[['September 26, 2022', '00:52', '01:10', '01:37', None, None],
 ['September 27, 2022', '00:37', '01:01', '00:36', None, None]]

In [70]:
def min_to_sec(time):
    if time:
        mins,secs = [int(i) for i in time.split(':')]
        return int(mins * 60 + secs)
    else:
        return time

In [64]:
min_to_sec('2:54')

174

In [76]:
# data = pd.DataFrame(columns=['date'] + usernames,data=rows)
data = pd.DataFrame(columns=['date'] + codes,data=rows)
data.head()

,date,gnome,andyvo,speckled mug,jack-o-pie,megh
0,"September 26, 2022",00:52,01:10,01:37,None,None
1,"September 27, 2022",00:37,01:01,00:36,None,None
2,"September 28, 2022",00:40,00:56,00:32,None,None
3,"September 29, 2022",01:07,01:08,01:27,None,None
4,"October 1, 2022",08:27,04:13,03:31,None,None


In [72]:
# convert time data in seconds
for c in data.columns[1:]:
    data[c] = pd.Series(data[c].apply(lambda x:min_to_sec(x)), dtype="Int64")
data.head()

,date,gnome,andyvo,speckled mug,jack-o-pie,megh
0,"September 26, 2022",52,70,97,<NA>,<NA>
1,"September 27, 2022",37,61,36,<NA>,<NA>
2,"September 28, 2022",40,56,32,<NA>,<NA>
3,"September 29, 2022",67,68,87,<NA>,<NA>
4,"October 1, 2022",507,253,211,<NA>,<NA>


In [73]:
data.to_csv('mini_scores_wide.csv',index=False)

In [74]:
# convert to long
long_data = pd.melt(data,id_vars='date',value_vars=data.columns[1:],var_name='user',value_name='time')
long_data.head()

,date,user,time
0,"September 26, 2022",gnome,52
1,"September 27, 2022",gnome,37
2,"September 28, 2022",gnome,40
3,"September 29, 2022",gnome,67
4,"October 1, 2022",gnome,507


In [75]:
long_data.to_csv('mini_scores_long.csv',index=False)